In [1]:
from pyspark.sql import SparkSession

In [27]:
MAX_MEMORY = "5g"
spark = SparkSession.builder.appName("taxi-duration-prediction-2")\
            .config("spark.executor.memory", MAX_MEMORY)\
            .config("spark.driver.memory", MAX_MEMORY).getOrCreate()

In [28]:
trip_files = "/Users/keon/fastcampus/data-engineering/01-spark/data/trips/*"

In [29]:
trips_df = spark.read.csv(f"file:///{trip_files}", inferSchema=True, header=True)

In [30]:
trips_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [31]:
trips_df.createOrReplaceTempView("trips")

In [32]:
query = """
SELECT
    passenger_count,
    PULocationID as pickup_location_id,
    DOLocationID as dropoff_location_id,
    trip_distance,
    HOUR(tpep_pickup_datetime) as pickup_time,
    DATE_FORMAT(TO_DATE(tpep_pickup_datetime), 'EEEE') AS day_of_week,
    total_amount
FROM
    (SELECT
        *,
        TO_DATE(t.tpep_pickup_datetime) AS pickup_date
    FROM
        trips t)
WHERE
    total_amount < 5000
    AND total_amount > 0
    AND trip_distance > 0
    AND trip_distance < 500
    AND passenger_count < 4
    AND pickup_date >= '2021-01-01'
    AND pickup_date < '2021-08-01'
"""
data_df = spark.sql(query)
data_df.createOrReplaceTempView("data")

In [33]:
data_df.printSchema()

root
 |-- passenger_count: integer (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)



In [34]:
data_df.select(["passenger_count", "trip_distance", "pickup_time", "day_of_week"]).describe().show()

+-------+------------------+------------------+------------------+-----------+
|summary|   passenger_count|     trip_distance|       pickup_time|day_of_week|
+-------+------------------+------------------+------------------+-----------+
|  count|          13126040|          13126040|          13126040|   13126040|
|   mean|1.2114008489993935|2.8820930920521075|14.206110144415224|       null|
| stddev|0.5424025151958388|3.8203064806711784|  5.11809582930429|       null|
|    min|                 0|              0.01|                 0|     Friday|
|    max|                 3|             475.5|                23|  Wednesday|
+-------+------------------+------------------+------------------+-----------+



In [35]:
train_df, test_df = data_df.randomSplit([0.8, 0.2], seed=1)
print(train_df.count())
print(test_df.count())

10500134
2625906


In [36]:
toy_df = train_df.sample(False, .1, seed=261)

In [38]:
# 컬럼 기반 포멧 parquet로 저장.. 압축률이 좋고 disk io가 적다 컬럼별로 적절한 인코딩이 가능
data_dir = "/Users/keon/fastcampus/data-engineering/01-spark/data"
train_df.write.format("parquet").save(f"{data_dir}/train/")
test_df.write.format("parquet").save(f"{data_dir}/test/")
toy_df.write.format("parquet").save(f"{data_dir}/toy/")

In [39]:
# 다시 읽어오기
train_df = spark.read.parquet(f"{data_dir}/train/")
test_df = spark.read.parquet(f"{data_dir}/test/")
toy_df = spark.read.parquet(f"{data_dir}/toy/")

In [40]:
train_df.printSchema()

root
 |-- passenger_count: integer (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)



In [41]:
# One-Hot Encoding
from pyspark.ml.feature import OneHotEncoder, StringIndexer

# 카테고리 피쳐들
cat_feats = [
    "pickup_location_id",
    "dropoff_location_id",
    "day_of_week",
    "pickup_time",
]

# 파이프라인 스테이지
stages = []

# 카테고리 피쳐 프리프로세싱
for c in cat_feats:
    # c -> c_idx
    cat_indexer = StringIndexer(inputCol=c, outputCol = c + "_idx").setHandleInvalid("keep")
    # one hot encode 
    onehot_encoder = OneHotEncoder(inputCols=[cat_indexer.getOutputCol()], outputCols=[c + "_onehot"])
    stages += [cat_indexer, onehot_encoder]

In [42]:
# Feature Normalization
from pyspark.ml.feature import VectorAssembler, StandardScaler

# Numerical features
num_feats = [
    "passenger_count",
    "trip_distance"
]

# vector assembler
for n in num_feats:
    num_assembler = VectorAssembler(inputCols=[n], outputCol=n+"_vector")
    num_scaler = StandardScaler(inputCol=num_assembler.getOutputCol(), outputCol=n+"_scaled")
    stages += [num_assembler, num_scaler]

In [43]:
# Categorical + Numeric featurees
assembler_inputs = [c + "_onehot" for c in cat_feats] + [n + "_scaled" for n in num_feats]
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="feature_vector")
stages += [assembler]

In [44]:
from pyspark.ml import Pipeline
# add model into the stages
transform_stages = stages

# Construct pipeline using the set of stages defined
pipeline =  Pipeline(stages=transform_stages)

# Fit the transformer
fitted_transformer = pipeline.fit(train_df)

In [45]:
# Transform the train data
transformed_train_df = fitted_transformer.transform(train_df)
# transformed_train_df = transformed_train_df.cache()

In [51]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(maxIter=100, 
                         solver="normal", 
                         labelCol="total_amount",
                         featuresCol="feature_vector",
                        )

In [52]:

model = lr.fit(transformed_train_df)

In [53]:
from pyspark.ml.evaluation import RegressionEvaluator

In [54]:
transformed_test_df = fitted_transformer.transform(test_df)

In [55]:
predictions = model.transform(transformed_test_df).cache()

In [56]:
predictions.show()

+---------------+------------------+-------------------+-------------+-----------+-----------+------------+----------------------+-------------------------+-----------------------+--------------------------+---------------+------------------+---------------+------------------+----------------------+----------------------+--------------------+--------------------+--------------------+------------------+
|passenger_count|pickup_location_id|dropoff_location_id|trip_distance|pickup_time|day_of_week|total_amount|pickup_location_id_idx|pickup_location_id_onehot|dropoff_location_id_idx|dropoff_location_id_onehot|day_of_week_idx|day_of_week_onehot|pickup_time_idx|pickup_time_onehot|passenger_count_vector|passenger_count_scaled|trip_distance_vector|trip_distance_scaled|      feature_vector|        prediction|
+---------------+------------------+-------------------+-------------+-----------+-----------+------------+----------------------+-------------------------+-----------------------+--------

In [ ]:
from pyspark.sql.types import DoubleType
distance_list = [1.1, 5.5, 10.5, 30.0]
distances_df = spark.createDataFrame(distance_list, DoubleType()).toDF("trip_distance")

In [ ]:
distances_df.show()

In [ ]:
vdistances_df = vassembler.transform(distances_df)

In [ ]:
vdistances_df.show()

In [ ]:
model.transform(vdistances_df).show()

# 성능 평가

In [57]:
model.summary

In [58]:
print("RMSE: ", model.summary.rootMeanSquaredError)

RMSE:  5.583560055498585


In [59]:
print("R2: ", model.summary.r2)
# R2:  0.018565176935511962

R2:  0.8126552959425991


In [60]:
train_df.describe().show()

+-------+------------------+------------------+-------------------+------------------+-----------------+-----------+------------------+
|summary|   passenger_count|pickup_location_id|dropoff_location_id|     trip_distance|      pickup_time|day_of_week|      total_amount|
+-------+------------------+------------------+-------------------+------------------+-----------------+-----------+------------------+
|  count|          10500134|          10500134|           10500134|          10500134|         10500134|   10500134|          10500134|
|   mean|1.2113262554554065| 166.2091519974888| 163.70784829984075|2.8818981624425497|14.20592756244825|       null| 17.97347660814364|
| stddev|0.5424053686238297| 65.92906003030252|  70.61811328008771| 3.807693411355345|5.118674976578861|       null|12.900023515121251|
|    min|                 0|                 1|                  1|              0.01|                0|     Friday|              0.01|
|    max|                 3|               265| 